In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\Potato-disease-end-to-end\\research'

In [2]:
import os

os.chdir("../")
%pwd

'c:\\Users\\44787\\Desktop\\projects\\Potato-disease-end-to-end'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainingConfig:
    root_dir: Path
    dataset_path: Path
    model_save: Path
    BATCH_SIZE: int
    IMAGE_SIZE: int
    CHANNELS: int
    EPOCHS: int

In [4]:
from PotatoDisease.constants import PARAMS_PATH, CONFIG_PATH
from PotatoDisease.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config=CONFIG_PATH,
                 params=PARAMS_PATH) -> None:
        self.config = read_yaml(config)
        self.params = read_yaml(params)

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            dataset_path=config.dataset_path,
            model_save=config.model_save,
            BATCH_SIZE=self.params.BATCH_SIZE,
            IMAGE_SIZE=self.params.IMAGE_SIZE,
            CHANNELS=self.params.CHANNELS,
            EPOCHS=self.params.EPOCHS,
        )
        return model_training_config

In [6]:
from PotatoDisease.logging import logger
import tensorflow as tf

[2023-12-18 08:19:57,081: WARNING: module_wrapper: From c:\Users\44787\anaconda3\envs\potato-env\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [7]:
class ModelTraining:
    def __init__(self, config):
        self.config = config

    def split_dataset(self,
                      dataset,
                      train_split=0.8,
                      test_split=0.1,
                      val_split=0.1,
                      shuffle=True):

        assert (train_split+test_split+val_split) == 1

        dataset_size = len(dataset)
        if shuffle:
            dataset = dataset.shuffle(10000, seed=42)

        train_size = int(train_split * dataset_size)
        val_size = int(val_split * dataset_size)

        train_data = dataset.take(train_size)
        val_data = dataset.skip(train_size).take(val_size)
        test_data = dataset.skip(train_size).skip(val_size)

        return train_data, val_data, test_data

    def prepare_dataset(self):
        dataset = tf.keras.preprocessing.image_dataset_from_directory(
            self.config.dataset_path,
            image_size=(self.config.IMAGE_SIZE, self.config.IMAGE_SIZE),
            batch_size=self.config.BATCH_SIZE,
            shuffle=True
        )
        logger.info(f'name of classes: {dataset.class_names}')

        train_data, val_data, test_data = self.split_dataset(dataset)

        train_data = train_data.cache().shuffle(
            1000).prefetch(buffer_size=tf.data.AUTOTUNE)
        val_data = val_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
        test_data = test_data.cache().shuffle(
            1000).prefetch(buffer_size=tf.data.AUTOTUNE)

        # applay data agumentation to train dataset
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.RandomFlip(
                "horizontal_and_vertical"),
            tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        ])
        train_data = train_data.map(lambda x, y:
                                    (data_augmentation(x, training=True), y)).prefetch(buffer_size=tf.data.AUTOTUNE)

        return train_data, val_data, test_data

    def training(self):
        resize_and_rescale = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.Resizing(
                self.config.IMAGE_SIZE, self.config.IMAGE_SIZE),
            tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
        ])

        # create a model architecture
        input_shape = (self.config.BATCH_SIZE, self.config.IMAGE_SIZE,
                       self.config.IMAGE_SIZE, self.config.CHANNELS)
        n_classes = 3

        model = tf.keras.models.Sequential([
            resize_and_rescale,
            tf.keras.layers.Conv2D(32, kernel_size=(
                3, 3), activation='relu', input_shape=input_shape),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64,  kernel_size=(3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64,  kernel_size=(3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(n_classes, activation='softmax'),
        ])
        model.build(input_shape=input_shape)
        logger.info(model.summary())

        # compile the model
        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=False),
            metrics=['accuracy']
        )

        train_data, val_data, test_data = self.prepare_dataset()

        # train the model
        history = model.fit(
            train_data,
            batch_size=self.config.BATCH_SIZE,
            validation_data=val_data,
            verbose=1,
            epochs=self.config.EPOCHS
        )

        # evaluate the model
        results = model.evaluate(test_data)
        logger.info(f"Model evaluation: {results}")

        self.save_model(
            self.config.model_save,
            model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [8]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(model_training_config)
    model_training.training()
except Exception as e:
    raise e

[2023-12-18 08:19:58,428: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-18 08:19:58,431: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-18 08:19:58,433: INFO: common: created directory at: artifacts/model_training]
[2023-12-18 08:19:58,699: WARNING: module_wrapper: From c:\Users\44787\anaconda3\envs\potato-env\lib\site-packages\keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.
]
[2023-12-18 08:19:58,803: WARNING: module_wrapper: From c:\Users\44787\anaconda3\envs\potato-env\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                      

c:\Users\44787\anaconda3\envs\potato-env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
